<a href="https://colab.research.google.com/github/StevenJGilb/stevenjgilb.github.io/blob/main/SQL_Murder_Mystery_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Setting Up**

For the initial set up, I need to:
* import sqlite3 in order to interact with the database
* import pandas to display the result  
* connect to database that I have uploaded in my Google Drive

In [3]:
import pandas as pd
import sqlite3 as sql

In [4]:
#Connecting to the database in my Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
conn = sql.connect('/content/drive/MyDrive/Colab Notebooks/sql-murder-mystery.db')


Mounted at /content/drive


In [5]:
cursor = conn.cursor()

In [6]:
#Set the dataframe width to max for better viewing.
pd.set_option("display.max_colwidth", None)

# **STARTING POINT**

"A crime has taken place and the detective needs your help. Bitch

The detective gave you the crime scene report, but you somehow lost it.

You vaguely remember that **the crime was a murder** that occurred sometime on **Jan.15, 2018** and that it **took place in SQL City**.

Start by retrieving the corresponding crime scene report from the police department's database."

Clue from website http://mystery.knightlab.com/


In [7]:
#Find the crime scene report for the murder
query_1 = """
SELECT *
FROM crime_scene_report
WHERE
    type LIKE 'Murder' AND
    date LIKE '20180115' AND
    city LIKE 'SQL City'
;
"""

#Read the SQL query
pd.read_sql_query(query_1,conn)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


# **First Witness**
Finding the first witness with these criteria:
*   Lives on 'Northwestern Dr'
*   Lives at the last house


In [ ]:
#First witness
query_2 = """
SELECT *
FROM person
WHERE
    address_street_name LIKE 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1
;
"""

#Read the SQL query
pd.read_sql_query(query_2,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


#**Second Witness**
Finding the first witness with these criteria:

*   Named Annabel
*   Lives somewhere on 'Franklin Ave'

In [ ]:
query_3 = """
SELECT *
FROM person
WHERE
    name LIKE '%Annabel%' AND
    address_street_name LIKE 'Franklin Ave'
;
"""

#Read the SQL query
pd.read_sql_query(query_3,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


#**Witness Information**
Witness ID obtained:
*   14887
*   16371

Next, we need to find their interview transcript of the witnesses


In [ ]:
query_4 = """
SELECT *
FROM interview
WHERE person_id IN (14887, 16371)
;
"""

#Read the SQL query
pd.read_sql_query(query_4,conn)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


#**4 Key Information of the Suspect**
Clues from both witnesses lead to "Get Fit Now Gym".

Key information:
1.   Membership id started with "48Z"
2.   Happened on January 9th
3.   Car plate included "H42W"
4.   Has gold member status

Key information #1 & #2 can be found in table "*get_fit_now_member*"

I will put aside key information #3, as it has no relation to Get Fit Now Gym

Key information #4 can be found in table "*get_fit_now_check_in*"

Let's look at key information #1 & #2

In [ ]:
query_5 = """
SELECT *
FROM get_fit_now_check_in
WHERE
    membership_id LIKE '48Z%' AND
    check_in_date LIKE '20180109'
;
"""

#Read the SQL query
pd.read_sql_query(query_5,conn)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


#**Result from Key Information #1 (*membership_id*) and #2 (*check_in_date*)**
*membership_id*:
* 48Z7A
* 48Z55

Now we got 2 suspects who matches the key information #1 & #2

As mentioned, I will put aside key information #3 (*plate_number*) and proceed querying based on key information #4 (*status*)

In [ ]:
query_6 = """
SELECT *
FROM get_fit_now_member
WHERE
    id LIKE '48Z%' AND
    membership_status = 'gold'
;
"""

#Read the SQL query
pd.read_sql_query(query_6,conn)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


#**Result from Key Information #4 (*status*)**
*id* (*membership_id*):
*   48Z7A
*   48Z55

Despite querying with different criteria, we received the same result as the result from key information #1 & #2

However, we found out more information about these 2 suspects, which are their *name* and *person_id*


Let's look at key information 3, hoping we can find more information

I will try to search based on the *plate_number* from table "*drivers_license*" from the 2 suspects

In [ ]:
query_7 = """
SELECT *
FROM drivers_license
WHERE
    plate_number LIKE '%H42W%'
;
"""

#Read the SQL query
pd.read_sql_query(query_7,conn)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


#**Result from Key Information #3 (plate_number)**

*id (license_id)*
*   183779
*   423327
*   664760

Now we got the *license_id* of the suspects, I have to match *license_id* with *person_id* from table "*person*"

In [ ]:
query_8 = """
SELECT *
FROM person
WHERE
    license_id IN ('183779', '423327', '664760') AND
    id IN ('28819', '67318')
;
"""

#Read the SQL query
pd.read_sql_query(query_8,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


#**One Suspect Left...**
We finally found the one suspect, Jeremy Bowers!

Let's check with the table "*solution*"!

In [ ]:
#Input our suspect's name
query_9 = """
INSERT INTO
  solution VALUES (1, 'Jeremy Bowers')
;
"""
cursor.execute(query_9)

In [ ]:
#To view the result
query_10 = """
  SELECT value FROM solution;
"""

#Read the SQL query
pd.read_sql_query(query_10,conn)

,value
0,"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


#**Murderer Found! (But Wait...)**
We finally found the murderer! However, the creator seems to have something more to this mystery, saying that there is **a real villain** and to interview the murderer

***Note: there is a 2 query limit challenge to find out the real villain,
however let's do the easy way for now***

Let's continue our mystery journey by query Jeremy Bowers' interview transcript

In [ ]:
query_11 = """
SELECT *
FROM interview
WHERE person_id LIKE '67318'
;
"""

#Read the SQL query
pd.read_sql_query(query_11,conn)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


#**New Clues!**
Jeremy Bowers confesses that he was hired by a woman with these characteristics:
1.  Height around 5'5" (65"") or 5'7" (67"")
2.  Has red hair
3.  Drives a Tesla Model S
4.  Attended SQL Symphony Concert 3 times in December 2017

My first thought is to look into characteristic #4, as less people are likely to attend an event 3 times a month (December 2017).

We can find that information in table "*facebook_event_checkin*"

In [ ]:
query_12 = """
SELECT *
FROM facebook_event_checkin
WHERE
    event_name LIKE 'SQL Symphony Concert' AND
    date LIKE '201712%'
GROUP BY
    person_id
HAVING
    COUNT(person_id) = 3
ORDER BY
  person_id
;
"""

#Read the SQL query
pd.read_sql_query(query_12,conn)


,person_id,event_id,event_name,date
0,24556,1143,SQL Symphony Concert,20171207
1,99716,1143,SQL Symphony Concert,20171206


#**Result from Characteristic #4 (Attend SQL Symphony Concert 3 Times)**

*person_id*
*   24556
*   99716

From characteristic #4, we found out there were only 2 people who attended the SQL Symphony Concert 3 times on December 2017 and the suspects' *person_id*.

Next, the characteristics #1 - #3 can be found in table "*drivers_license*", however we need *license_id* first!

In order to find the suspects' *license_id*, we can search through *person_id* in table "*person*"


In [ ]:
query_13 = """
SELECT *
FROM person
WHERE
    id IN ('24556', '99716')
;
"""

#Read the SQL query
pd.read_sql_query(query_13,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,24556,Bryan Pardo,101191,703,Machine Ln,816663882
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


#**More Information About the 2 Suspects**

From table "*person*", we now know the 2 suspects' name and license_id:
*   *name* = Bryan Pardo, *license_id* = 101191
*   *name* = Miranda Priestly, *license_id* = 202298

From here, we can match characteristics #1 - #3 with table "*drivers_license*" based on *license_id* we got from previous query.

As a reminder, these are our suspect's characteristics checklist:
*   Height around 65"" and 67""
*   Hair color red
*   Car make and model Tesla Model S

In [ ]:
query_14 = """
SELECT *
FROM drivers_license
WHERE
    id IN ('101191', '202298') AND
    hair_color LIKE 'red' AND
    height BETWEEN 65 AND 67
;
"""

#Read the SQL query
pd.read_sql_query(query_14,conn)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S


#**Real Villain Found!**

Through the process of filtering and elimination, we finally have one suspect with *license_id* 202298.
Based on our previous query (query_13), we know that license_id is owned by...

\*drum roll please...*

**MIRANDA PRIESTLY!**

We can now check the result in table "*solution*"

In [10]:
#Input our suspect's name
query_15 = """
INSERT INTO
  solution VALUES (1, 'Miranda Priestly')
;
"""
cursor.execute(query_15)

In [11]:
query_16 = """
SELECT value
FROM solution
;
"""
#Read the SQL query
pd.read_sql_query(query_16,conn)

,value
0,"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"


#**Challenge: Accepted!**

Continuing from the first solution check, the creator's challenge is to limit the number of queries to 2 (See "Murderer Found! But Wait...")

My plan of using the 2 queries are as follow:
1.    Receive the interview transcript from Jeremy Bowers (We have done that already, so I am going to skip this step)
2.    Find the suspect's name based on the characteristics given by Jeremy Bowers

Note: I will exclude checking the solution as part of the challenge, as it was not part of the challenge.

As a reminder, these are the characteristics of the real villain:
1.  Height around 5'5" (65"") or 5'7" (67"")
2.  Has red hair
3.  Drives a Tesla Model S
4.  Attended SQL Symphony Concert 3 times in December 2017

And I will be attempting finding the real villain in 1 query through the sub-query method

In [15]:
query_17 = """
SELECT *
FROM person p
WHERE p.license_id IN (
    SELECT d.id
    FROM drivers_license d
    WHERE
        height BETWEEN 65 AND 67 AND
        hair_color LIKE 'red' AND
        car_make LIKE 'Tesla' AND
        car_model LIKE 'Model S' AND
        d.id IN (
            SELECT license_id
            FROM person p
            WHERE p.id IN (
                SELECT person_id
                FROM facebook_event_checkin
                WHERE
                    date LIKE '201712%' AND
                    event_name LIKE 'SQL Symphony Concert'
                GROUP BY person_id
                HAVING COUNT(person_id) = 3
                )
           )
    )
;
"""

#Read the SQL query
pd.read_sql_query(query_17,conn)


,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


#**Challenge: Completed!**

Through sub-query method, we were able to have the same result as what we did previously (3 queries) in 1 query.

